![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/util/OpenAICompletion.ipynb)

## OpenAICompletion in SparkNLP

In this notebook, we'll explore the process of utilizing OpenAICompletition within SparkNLP's framework.

Spark NLP offers a seamless integration with various OpenAI APIs, presenting a powerful synergy. With the introduction of Spark NLP 5.1.0, leveraging the OpenAICompletition and OpenAIEmbeddings transformers becomes achievable. This integration not only ensures the utilization of OpenAI's capabilities but also capitalizes on Spark's inherent scalability advantages.

## Spark NLP Settings

All you need to do is to setup your [OpenAI API Key](https://platform.openai.com/docs/api-reference/authentication) and add it to Spark properties

In [ ]:
print("Enter your OPENAI API Key:")
OPENAI_API_KEY = input()

In [ ]:
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from sparknlp.base import LightPipeline

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
openai_params = {"spark.jsl.settings.openai.api.key": OPENAI_API_KEY}
spark = sparknlp.start(params=openai_params)

In [ ]:
document_assembler = DocumentAssembler() \
        .setInputCol("text") \
        .setOutputCol("document")

openai_completion = OpenAICompletion() \
       .setInputCols("document") \
       .setOutputCol("completion") \
       .setModel("text-davinci-003") \
       .setMaxTokens(50)

# Define the pipeline
pipeline = Pipeline(stages=[
    document_assembler, openai_completion
])

In [ ]:
empty_df = spark.createDataFrame([[""]], ["text"])
sample_text= [["Generate a restaurant review."], ["Write a review for a local eatery."], ["Create a JSON with a review of a dining experience."]]
sample_df= spark.createDataFrame(sample_text).toDF("text")
sample_df.show()

+--------------------+
|                text|
+--------------------+
|Generate a restau...|
|Write a review fo...|
|Create a JSON wit...|
+--------------------+



In [ ]:
pipeline_model = pipeline.fit(empty_df)
completion_df = pipeline_model.transform(sample_df)

In [ ]:
completion_df.select("completion").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|completion                                                                                                                                                                                                                                                                                      |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{document, 0, 256, \n\nI had the pleasure of eating at [name of restaurant] recently and I was thoroughly impressed. The atmo

LightPipeline

In [ ]:
light_pipeline_openai = LightPipeline(pipeline_model)

In [ ]:
light_pipeline_openai.fullAnnotate("Generate a negative review of a movie")

[{'document': [Annotation(document, 0, 36, Generate a negative review of a movie, {}, [])],
  'completion': [Annotation(document, 0, 235, 
   
   This movie was terrible! The plot was extremely contrived and the acting was terrible. It was just such a waste of time and money. The dialogue was especially bad and the special effects were really cheap. Even if you try to watch it, {}, [])]}]

In [ ]:
light_pipeline_openai.annotate("Generate a negative review of a movie")

{'document': ['Generate a negative review of a movie'],
 'completion': ['\n\nThis movie was terrible! Despite the star-studded cast, the story was a jumbled mess that made no sense. The characters had no motivation and the dialogue was pretty stiff and unbelievable. The special effects were way over the top']}